In [4]:
using DrWatson
@quickactivate "HEMI" 
using HEMI 
using DataFrames, Chain
using Plots

In [16]:
## Directorios de resultados 
config_savepath = datadir("results", "corr-combination", "Esc-F")
config_savepath_absme = datadir("results", "absme-combination", "Esc-G")
tray_dir = datadir(config_savepath, "tray_infl")

# Directorios de resultados de combinación MAI 
maioptfn_path = datadir("results", "CoreMai", "Esc-F", "BestOptim", "corr-weights", "maioptfn.jld2")

# CountryStructure con datos hasta período de evaluación 
gtdata_eval = gtdata[Date(2020, 12)]

UniformCountryStructure{2, Float32, Float32} con 2 bases
|─> VarCPIBase{Float32, Float32}: 120 períodos × 218 gastos básicos Jan-2001-Dec-2010
|─> VarCPIBase{Float32, Float32}: 120 períodos × 279 gastos básicos Jan-2011-Dec-2020


In [13]:
df_results = collect_results(config_savepath)
df_results

┌ Info: Scanning folder c:\Users\RRCP\Documents\HEMI\HEMI\data\results\corr-combination\Esc-F for result files.
└ @ DrWatson C:\Users\RRCP\.julia\packages\DrWatson\VwP8P\src\result_collection.jl:88


┌ Info: Added 7 entries.
└ @ DrWatson C:\Users\RRCP\.julia\packages\DrWatson\VwP8P\src\result_collection.jl:118


,gt_b00_mse_bias,traindate,inflfn
,Float32?,Date?,Inflati…?
1,0.0384297,2020-12-01,"CombinationFunction{3, Vector{Float32}}(EnsembleFunction{3}((InflationCoreMai{Float32, Float64, MaiG{Vector{Float64}}}(-200.0f0:0.01f0:200.0f0, MaiG{Vector{Float64}}(3, [0.0, 0.323195, 0.77172, 1.0])), InflationCoreMai{Float32, Float64, MaiFP{Vector{Float64}}}(-200.0f0:0.01f0:200.0f0, MaiFP{Vector{Float64}}(5, [0.0, 0.335389, 0.65647, 0.781121, 0.860586, 1.0])), InflationCoreMai{Float32, Float64, MaiF{Vector{Float64}}}(-200.0f0:0.01f0:200.0f0, MaiF{Vector{Float64}}(5, [0.0, 0.318405, 0.656481, 0.777264, 0.871024, 1.0])))), Float32[0.00581682, 0.434901, 0.55931], ""MAI óptima de correlación 2018"")"
2,3.15955,2020-12-01,"InflationDynamicExclusion(0.468323, 4.97451)"
3,0.107584,2020-12-01,"InflationFixedExclusionCPI{2}(([35, 30, 190, 36, 37, 40, 31, 104, 162, 32, 33, 159], [29, 116, 31, 46, 39, 40, 186, 30, 35, 185, 197, 34, 48, 184, 41, 47, 37, 22, 25, 229, 38, 32, 274, 3, 45, 44, 33, 237, 19, 10, 24, 275, 115, 15, 59, 42, 61, 43, 113, 49, 27, 71, 23, 268, 9, 36, 236, 78, 20, 213, 273, 26]))"
4,5.03112,2020-12-01,"InflationTrimmedMeanEq(55.9051, 92.1777)"
5,44.0879,2020-12-01,"InflationTrimmedMeanWeighted(46.4432, 98.5461)"
6,4.26757,2020-12-01,InflationPercentileEq(0.772522)
7,33.4896,2020-12-01,InflationPercentileWeighted(0.809557)


## Métricas de los 4 períodos

In [12]:
select(df_results, :measure, r"corr")

,measure,gt_b00_corr,gt_t0010_corr,gt_b10_corr
,String?,Float32?,Float32?,Float32?
1,MAI óptima de correlación 2018,0.971953,0.980825,0.876439
2,"Inflación de exclusión dinámica (0.47, 4.97)",0.940207,0.972252,0.722675
3,"Exclusión fija de gastos básicos IPC (12, 52)",0.946636,0.969423,0.613
4,"Media Truncada Equiponderada (55.91, 92.18)",0.9722,0.978092,0.872161
5,"Media Truncada Ponderada (46.44, 98.55)",0.942932,0.972701,0.734026
6,Percentil equiponderado 77.25,0.968672,0.978261,0.871594
7,Percentil ponderado 80.96,0.932154,0.974553,0.756394


## Métricas de la combinación óptima CORR

In [70]:
optcorr2022 = wload(datadir(config_savepath, "optcorr2022", "optcorr2022.jld2"), "optcorr2022")
components_df = components(optcorr2022)

,measure,weights
,String,Float32
1,Percentil ponderado 80.96,3.93042e-5
2,"Inflación de exclusión dinámica (0.47, 4.97)",0.00194286
3,"Media Truncada Ponderada (46.44, 98.55)",0.0
4,Percentil equiponderado 77.25,0.19322
5,MAI óptima de correlación 2018,0.336925
6,"Media Truncada Equiponderada (55.91, 92.18)",0.467873
7,"Exclusión fija de gastos básicos IPC (12, 52)",0.0


In [72]:
# DataFrame de combinación 
combine_df = @chain df_results begin 
    select(:measure, :corr, :me, :inflfn, 
        :path => ByRow(p -> joinpath(tray_dir, basename(p))) => :tray_path)
    sort(:corr)
end
combine_df = leftjoin(combine_df, components_df, on = :measure)
combine_df

## Obtener las trayectorias de los archivos guardados en el directorio tray_infl 
# Genera un arreglo de 3 dimensiones de trayectorias (T, n, K)
tray_infl = mapreduce(hcat, combine_df.tray_path) do path
    load(path, "tray_infl")
end

functions = combine_df.inflfn
components_mask = [!(fn isa InflationFixedExclusionCPI) for fn in functions];

In [74]:
# Obtener el parámetro
resamplefn = df_results[1, :resamplefn]
trendfn = df_results[1, :trendfn]
paramfn = df_results[1, :paramfn]
param = InflationParameter(paramfn, resamplefn, trendfn)
tray_infl_pob = param(gtdata_eval);

In [75]:
# Pesos de la combinación óptima
corr_weights = combine_df.weights

EVAL_PERIODS = [GT_EVAL_B00, GT_EVAL_T0010, GT_EVAL_B10, CompletePeriod()]

comb_metrics = mapreduce(merge, EVAL_PERIODS) do period
    # Ventana de evaluación
    eval_window = eval_periods(gtdata_eval, period)
    # Trayectorias combinadas
    tray_infl_opt = sum(tray_infl[eval_window, :, :] .* corr_weights', dims=2)
    metrics = eval_metrics(tray_infl_opt, tray_infl_pob[eval_window], prefix = period_tag(period))
end;


In [76]:
# Obtener un DataFrame de métricas de la combinación lineal CORR
comb_metrics_df = DataFrame(comb_metrics)
comb_metrics_df[!, :measure] .= measure_name(optcorr2022)
select(comb_metrics_df, :measure, r"corr")

,measure,corr,gt_b00_corr,gt_b10_corr,gt_t0010_corr
,String,Float32,Float32,Float32,Float32
1,Subyacente óptima CORR 2022,0.987285,0.972561,0.876094,0.979512
